Cloning into 'recsys'...
remote: Enumerating objects: 1895, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 1895 (delta 13), reused 20 (delta 8), pack-reused 1854 (from 1)
Receiving objects: 100% (1895/1895), 52.25 MiB | 26.18 MiB/s, done.
Resolving deltas: 100% (1180/1180), done.


# INTRO

In [1]:
import os
from typing import Tuple, Callable, Dict, Optional, List

import pandas as pd
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

from Evaluation.Evaluator import EvaluatorHoldout

In [2]:
URM = pd.read_csv("./data/data_train.csv")
ICM = pd.read_csv("./data/data_ICM_metadata.csv")

In [3]:
unique_users = URM.user_id.unique()
max_user_id = unique_users.max()
min_user_id = unique_users.min()
num_users = unique_users.size
print(f"Max User ID: {max_user_id}, Min User ID: {min_user_id}, Number of IDs: {num_users}")

unique_items = ICM.item_id.unique()
max_item_id = ICM.item_id.max()
min_item_id = ICM.item_id.min()
num_items = unique_items.size
unique_features = ICM.feature_id.unique()
num_features = unique_features.size
min_features_id = ICM.feature_id.min()
max_features_id = ICM.feature_id.max()
print(f"Max Item ID: {max_item_id}, Min Item ID: {min_item_id}, Number of IDs: {num_items}")
print(f"Max Feature ID: {max_features_id}, Min Item ID: {min_features_id}, Number of IDs: {num_features}")

Max User ID: 35735, Min User ID: 0, Number of IDs: 35736
Max Item ID: 38120, Min Item ID: 0, Number of IDs: 38121
Max Feature ID: 94330, Min Item ID: 0, Number of IDs: 94331


In [4]:
print("URM")
print(URM.head())
print("\nICM")
print(ICM.head())

URM
   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0

ICM
   item_id  feature_id  data
0     2519           0   1.0
1     2642           0   1.0
2     3316           0   1.0
3     3400           0   1.0
4     3472           0   1.0


In [5]:
URM.user_id

0              0
1              0
2              0
3              0
4              0
           ...  
1764602    35735
1764603    35735
1764604    35735
1764605    35735
1764606    35735
Name: user_id, Length: 1764607, dtype: int64

In [6]:
def dataset_splits(ratings, num_users, num_items, validation_percentage: float, testing_percentage: float):
    seed = 1234

    # Construct the whole URM as a sparse matrix
    urm_all = sp.csr_matrix((ratings.data, (ratings.user_id, ratings.item_id)),
                            shape=(num_users, num_items))

    # Split into train + validation and test sets
    train_val_indices, test_indices = train_test_split(
        np.arange(len(ratings)),
        test_size=testing_percentage,
        shuffle=True,
        random_state=seed
    )
    
    # Split train + validation into train and validation
    train_indices, val_indices = train_test_split(
        train_val_indices,
        test_size=validation_percentage / (1 - testing_percentage),
        shuffle=True,
        random_state=seed
    )
    
    # Get user, item, and rating data for each set
    train_data = ratings.iloc[train_indices]
    val_data = ratings.iloc[val_indices]
    test_data = ratings.iloc[test_indices]
    
    # Construct sparse matrices
    urm_train = sp.csr_matrix((train_data.data, (train_data.user_id, train_data.item_id)), 
                              shape=(num_users, num_items))
    urm_validation = sp.csr_matrix((val_data.data, (val_data.user_id, val_data.item_id)), 
                                   shape=(num_users, num_items))
    urm_test = sp.csr_matrix((test_data.data, (test_data.user_id, test_data.item_id)), 
                             shape=(num_users, num_items))

    return urm_all, urm_train, urm_validation, urm_test


In [7]:
urm_all, urm_train, urm_validation, urm_test = dataset_splits(URM, 
                                                     num_users=num_users, 
                                                     num_items=num_items, 
                                                     validation_percentage=0.10, 
                                                     testing_percentage=0.20)

In [8]:
icm_matrix = sp.csr_matrix((ICM.data, (ICM.item_id, ICM.feature_id)), 
                           shape=(num_items, num_features))


# Top Popular For Reference

In [12]:
class TopPopularRecommender:
    """Top Popular Recommender"""
    
    def __init__(self, URM_train):
        """
        Initialize the recommender with the training user-item matrix.

        Parameters:
        URM_train: Sparse matrix (csr) representing the training interactions.
        """
        self.URM_train = URM_train
        self.popular_items = None

    def fit(self):
        """Calculate the most popular items based on URM_train."""
        # Sum interactions for each item to get popularity
        item_popularity = np.array(self.URM_train.sum(axis=0)).flatten()
        
        # Get item indices sorted by popularity in descending order
        self.popular_items = np.argsort(-item_popularity)
    
    def recommend(self, user_id, top_k=10):
        """
        Recommend top-k popular items for a user.

        Parameters:
        user_id: The user ID for whom to recommend (not used here).
        top_k: Number of top popular items to return.

        Returns:
        List of top-k popular item IDs.
        """
        return self.popular_items[:top_k]


In [13]:
recommender = TopPopularRecommender(URM_train=urm_train)
recommender.fit()

In [15]:
# Initialize the evaluator with the test user-item matrix
evaluator = EvaluatorHoldout(urm_test, cutoff_list=[10] )

# Evaluate the TopPopularRecommender
results_df, results_run_string = evaluator.evaluateRecommender(recommender)

# Print the evaluation results
print(results_run_string)


EvaluatorHoldout: Ignoring 175 ( 0.5%) Users that have less than 1 test interactions


AttributeError: 'TopPopularRecommender' object has no attribute 'get_URM_train'

In [ ]:
def calculate_map_at_k(recommender, urm_test, target_user_ids, k=10):
    """
    Calculate MAP@K for a given recommender and test data.

    Parameters:
    recommender: The fitted recommender model.
    urm_test: Sparse matrix (csr) representing the test interactions.
    target_user_ids: List of user IDs to evaluate.
    k: The number of top recommendations to consider (e.g., 10 for MAP@10).

    Returns:
    mean_ap: The mean average precision at K across target users.
    """
    # List to store AP@K for each user
    average_precisions = []

    # Go through each target user with a progress bar
    for user_id in tqdm(target_user_ids, desc="Evaluating MAP@K"):
        # Get ground truth items for the user
        relevant_items = urm_test[user_id].indices

        # Generate recommendations and take the top-k
        recommended_items = recommender.recommend(user_id)[:k]

        # Calculate binary relevance
        relevance = np.isin(recommended_items, relevant_items).astype(int)

        # Calculate average precision for this user
        if relevance.sum() > 0:  # Only consider users with at least one relevant item in the test set
            ap = average_precision_score(relevance, np.ones(len(relevance)))
            average_precisions.append(ap)

    # Calculate mean AP@K
    mean_ap = np.mean(average_precisions) if average_precisions else 0.0
    return mean_ap




In [ ]:
# Use the list of target user IDs from data_target_users_test.csv
target_user_ids = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv")['user_id'].tolist()
map_at_10 = calculate_map_at_k(recommender, urm_test, target_user_ids, k=10)

print(f"MAP@10 for Top Popular Recommender: {map_at_10}")


# Graph Based Implementation

## P3alphaRecommender Import Test

In [11]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

In [19]:
recommender = P3alphaRecommender(URM_train=urm_train)
recommender.fit(topK=100, alpha=1.0, min_rating=0, implicit=True, normalize_similarity=True)

P3alphaRecommender: Similarity column 38121 (100.0%), 2350.31 column/sec. Elapsed time 16.22 sec


In [17]:
# Initialize the evaluator with the test user-item matrix
evaluator = EvaluatorHoldout(urm_test, cutoff_list=[10] )

# Evaluate the TopPopularRecommender
results_df, results_run_string = evaluator.evaluateRecommender(recommender)

# Print the evaluation results
print(results_run_string)

EvaluatorHoldout: Ignoring 175 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35561 (100.0%) in 30.66 sec. Users per second: 1160
CUTOFF: 10 - PRECISION: 0.0813222, PRECISION_RECALL_MIN_DEN: 0.1241966, RECALL: 0.1073040, MAP: 0.0393514, MAP_MIN_DEN: 0.0589435, MRR: 0.2356996, NDCG: 0.1181363, F1: 0.0925237, HIT_RATE: 0.4893844, ARHR_ALL_HITS: 0.3013392, NOVELTY: 0.0035830, AVERAGE_POPULARITY: 0.1494924, DIVERSITY_MEAN_INTER_LIST: 0.9941785, DIVERSITY_HERFINDAHL: 0.9994151, COVERAGE_ITEM: 0.5019805, COVERAGE_ITEM_HIT: 0.2047953, ITEMS_IN_GT: 0.9957504, COVERAGE_USER: 0.9951030, COVERAGE_USER_HIT: 0.4869879, USERS_IN_GT: 0.9951030, DIVERSITY_GINI: 0.1263450, SHANNON_ENTROPY: 12.3358126, RATIO_DIVERSITY_HERFINDAHL: 0.9994729, RATIO_DIVERSITY_GINI: 0.2082600, RATIO_SHANNON_ENTROPY: 0.8357690, RATIO_AVERAGE_POPULARITY: 2.0453177, RATIO_NOVELTY: 0.2677232, 



In [26]:
import optuna
from Evaluation.Evaluator import EvaluatorHoldout

# Define the number of trials
n_trials = 30

# Define the evaluator for validation
evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])

# Define the objective function as before
def objective(trial):
    topK = trial.suggest_int("topK", 10, 200)
    alpha = trial.suggest_float("alpha", 0.1, 1.5)
    min_rating = trial.suggest_int("min_rating", 0, 5)
    implicit = trial.suggest_categorical("implicit", [True, False])
    normalize_similarity = trial.suggest_categorical("normalize_similarity", [True, False])
    
    # Initialize and fit the recommender with the suggested hyperparameters
    recommender = P3alphaRecommender(URM_train=urm_train)
    recommender.fit(topK=topK, alpha=alpha, min_rating=min_rating, implicit=implicit, 
                    normalize_similarity=normalize_similarity)
    
    # Use EvaluatorHoldout to calculate MAP@10
    results_df, _ = evaluator_validation.evaluateRecommender(recommender)
    map_at_10 = results_df.loc[10]["MAP"]  # Extract MAP@10 from results
    
    return map_at_10

EvaluatorHoldout: Ignoring 1909 ( 5.3%) Users that have less than 1 test interactions


In [ ]:
# Create and run the Optuna study without the progress bar callback
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=n_trials)

# Output the best hyperparameters found
print("Best hyperparameters:", study.best_params)
print("Best MAP@10:", study.best_value)

In [28]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender


p3opt = P3alphaRecommender(URM_train=urm_train)
p3opt.fit(topK= 28, alpha= 0.10129039554581817, min_rating= 1, implicit= True, normalize_similarity= True)


P3alphaRecommender: Similarity column 38121 (100.0%), 2432.32 column/sec. Elapsed time 15.67 sec


In [30]:
# Initialize the evaluator with the test user-item matrix
evaluator = EvaluatorHoldout(urm_test, cutoff_list=[10] )

# Evaluate the TopPopularRecommender
results_df, results_run_string = evaluator.evaluateRecommender(p3opt)

# Print the evaluation results
print(results_run_string)

EvaluatorHoldout: Ignoring 175 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35561 (100.0%) in 32.77 sec. Users per second: 1085
CUTOFF: 10 - PRECISION: 0.0878519, PRECISION_RECALL_MIN_DEN: 0.1329276, RECALL: 0.1141491, MAP: 0.0437163, MAP_MIN_DEN: 0.0648082, MRR: 0.2515054, NDCG: 0.1275903, F1: 0.0992888, HIT_RATE: 0.5067630, ARHR_ALL_HITS: 0.3277530, NOVELTY: 0.0036988, AVERAGE_POPULARITY: 0.1055988, DIVERSITY_MEAN_INTER_LIST: 0.9976242, DIVERSITY_HERFINDAHL: 0.9997596, COVERAGE_ITEM: 0.5591669, COVERAGE_ITEM_HIT: 0.2519871, ITEMS_IN_GT: 0.9957504, COVERAGE_USER: 0.9951030, COVERAGE_USER_HIT: 0.5042814, USERS_IN_GT: 0.9951030, DIVERSITY_GINI: 0.1942625, SHANNON_ENTROPY: 13.1392412, RATIO_DIVERSITY_HERFINDAHL: 0.9998175, RATIO_DIVERSITY_GINI: 0.3202114, RATIO_SHANNON_ENTROPY: 0.8902024, RATIO_AVERAGE_POPULARITY: 1.4447768, RATIO_NOVELTY: 0.2763754, 



In [32]:
# Initialize the evaluator with the test user-item matrix
evaluator = EvaluatorHoldout(urm_test, cutoff_list=[10] )

# Evaluate the TopPopularRecommender
results_df, results_run_string = evaluator.evaluateRecommender(p3opt)

# Print the evaluation results
print(results_run_string)


EvaluatorHoldout: Ignoring 175 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35561 (100.0%) in 32.06 sec. Users per second: 1109
CUTOFF: 10 - PRECISION: 0.0878519, PRECISION_RECALL_MIN_DEN: 0.1329276, RECALL: 0.1141491, MAP: 0.0437163, MAP_MIN_DEN: 0.0648082, MRR: 0.2515054, NDCG: 0.1275903, F1: 0.0992888, HIT_RATE: 0.5067630, ARHR_ALL_HITS: 0.3277530, NOVELTY: 0.0036988, AVERAGE_POPULARITY: 0.1055988, DIVERSITY_MEAN_INTER_LIST: 0.9976242, DIVERSITY_HERFINDAHL: 0.9997596, COVERAGE_ITEM: 0.5591669, COVERAGE_ITEM_HIT: 0.2519871, ITEMS_IN_GT: 0.9957504, COVERAGE_USER: 0.9951030, COVERAGE_USER_HIT: 0.5042814, USERS_IN_GT: 0.9951030, DIVERSITY_GINI: 0.1942625, SHANNON_ENTROPY: 13.1392412, RATIO_DIVERSITY_HERFINDAHL: 0.9998175, RATIO_DIVERSITY_GINI: 0.3202114, RATIO_SHANNON_ENTROPY: 0.8902024, RATIO_AVERAGE_POPULARITY: 1.4447768, RATIO_NOVELTY: 0.2763754, 



In [12]:
p3opt = P3alphaRecommender(URM_train=urm_all)
p3opt.fit(topK= 28, alpha= 0.10129039554581817, min_rating= 1, implicit= True, normalize_similarity= True)


P3alphaRecommender: Similarity column 38121 (100.0%), 2216.98 column/sec. Elapsed time 17.19 sec


## RP3 beta

In [28]:
from Recommenders.GraphBased.RP3betaRecommenderICM import RP3betaRecommenderICM

In [29]:
# Initialize with both URM and ICM
recommender = RP3betaRecommenderICM(urm_train, icm_matrix)

# Fit with desired parameters
recommender.fit(
    alpha=1.0,    # Power for final normalization
    beta=0.6,     # Power for popularity scaling
    delta=0.5,    # Weight between URM and ICM paths
    topK=100      # Number of similar items to keep
)

RP3betaRecommenderICM: Normalized URM and ICM
RP3betaRecommenderICM: Processed 38121 ( 100.00% ) in 1.59 min. Items per second: 401
RP3betaRecommenderICM: Computation completed


In [13]:
# Initialize the evaluator with the test user-item matrix
evaluator = EvaluatorHoldout(urm_test, cutoff_list=[10] )

# Evaluate the TopPopularRecommender
results_df, results_run_string = evaluator.evaluateRecommender(recommender)

# Print the evaluation results
print(results_run_string)

EvaluatorHoldout: Ignoring 175 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35561 (100.0%) in 20.89 sec. Users per second: 1702
CUTOFF: 10 - PRECISION: 0.0730069, PRECISION_RECALL_MIN_DEN: 0.1118715, RECALL: 0.0970264, MAP: 0.0351426, MAP_MIN_DEN: 0.0529834, MRR: 0.2137484, NDCG: 0.1065018, F1: 0.0833202, HIT_RATE: 0.4468378, ARHR_ALL_HITS: 0.2711985, NOVELTY: 0.0039341, AVERAGE_POPULARITY: 0.0720026, DIVERSITY_MEAN_INTER_LIST: 0.9982356, DIVERSITY_HERFINDAHL: 0.9998208, COVERAGE_ITEM: 0.7736681, COVERAGE_ITEM_HIT: 0.2553186, ITEMS_IN_GT: 0.9957504, COVERAGE_USER: 0.9951030, COVERAGE_USER_HIT: 0.4446497, USERS_IN_GT: 0.9951030, DIVERSITY_GINI: 0.2988214, SHANNON_ENTROPY: 13.7400169, RATIO_DIVERSITY_HERFINDAHL: 0.9998786, RATIO_DIVERSITY_GINI: 0.4925605, RATIO_SHANNON_ENTROPY: 0.9309058, RATIO_AVERAGE_POPULARITY: 0.9851215, RATIO_NOVELTY: 0.2939597, 



In [67]:
import optuna
import numpy as np
from Evaluation.Evaluator import EvaluatorHoldout
import logging
import os
from datetime import datetime

def optimize_rp3beta_icm(urm_train, icm, urm_validation, n_trials=30, output_folder="optimization_results"):
    # Setup logging
    os.makedirs(output_folder, exist_ok=True)
    log_file = os.path.join(output_folder, f'optimization_log_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')
    
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    
    # Define the evaluator for validation
    evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
    
    def objective(trial):
        # Define hyperparameter search spaces
        params = params = {
            "topK": trial.suggest_int("topK", 10, 200),
            "alpha": trial.suggest_float("alpha", 0.1, 1.0),
            "beta": trial.suggest_float("beta", 0.1, 1.0),    
            "delta": trial.suggest_float("delta", 0.0, 1.0), 
            "min_rating": trial.suggest_float("min_rating", 0.0, 5.0),  
            "implicit": trial.suggest_categorical("implicit", [True, False]),
            "normalize_similarity": trial.suggest_categorical("normalize_similarity", [True, False])
        }
        
        try:
            # Initialize and fit the recommender
            recommender = RP3betaRecommenderICM(URM_train=urm_train, ICM=icm)
            recommender.fit(**params)
            
            # Calculate MAP@10
            results_df, _ = evaluator_validation.evaluateRecommender(recommender)
            map_at_10 = results_df.loc[10]["MAP"]
            
            # Log the trial results
            logging.info(f"Trial {trial.number}:")
            logging.info(f"Params: {params}")
            logging.info(f"MAP@10: {map_at_10}")
            
            return map_at_10
            
        except Exception as e:
            logging.error(f"Error in trial {trial.number}: {str(e)}")
            raise optuna.exceptions.TrialPruned()
    
    # Create and run the study
    study = optuna.create_study(direction="maximize")
    
    # Add logging callback
    def logging_callback(study, trial):
        if study.best_trial == trial:
            logging.info(f"New best trial (#{trial.number}) - MAP@10: {trial.value}")
            logging.info(f"Best hyperparameters: {trial.params}")
    
    try:
        study.optimize(objective, n_trials=n_trials, callbacks=[logging_callback])
        
        # Save the results
        results = {
            "best_params": study.best_params,
            "best_map": study.best_value,
            "n_trials": n_trials,
            "optimization_history": [
                {
                    "trial": t.number,
                    "value": t.value,
                    "params": t.params
                }
                for t in study.trials if t.value is not None
            ]
        }
        
        # Log final results
        logging.info("\nOptimization completed!")
        logging.info(f"Best MAP@10: {study.best_value}")
        logging.info(f"Best hyperparameters: {study.best_params}")
        
        # Create visualization plots
        try:
            from optuna.visualization import plot_optimization_history, plot_param_importances
            import matplotlib.pyplot as plt
            
            # Plot optimization history
            fig1 = plot_optimization_history(study)
            fig1.write_image(os.path.join(output_folder, "optimization_history.png"))
            
            # Plot parameter importances
            fig2 = plot_param_importances(study)
            fig2.write_image(os.path.join(output_folder, "param_importances.png"))
            
        except ImportError:
            logging.warning("Plotly not installed. Skipping visualization generation.")
        
        return results
        
    except Exception as e:
        logging.error(f"Optimization failed: {str(e)}")
        raise

In [68]:
if __name__ == "__main__":

    # Call the optimizer for SLIMElasticNet
    results = optimize_slim_elasticnet(
        urm_train=urm_train,
        urm_validation=urm_validation,
        n_trials=100  # Number of optimization trials
    )

    # Print best results
    print("Best Hyperparameters:")
    print(results["best_params"])
    print("Best NDCG@10:")
    print(results["best_ndcg"])


[I 2024-11-26 00:57:27,023] A new study created in memory with name: no-name-77b22015-edb4-4090-86e1-8bb513d34012
2024-11-26 00:57:27,027 - INFO - Starting optimization process...
2024-11-26 00:57:27,039 - INFO - 
Starting Trial 0
2024-11-26 00:57:27,042 - INFO - Parameters: {'alpha': 0.04216944928979198, 'l1_ratio': 0.874776361499749, 'topK': 218, 'workers': 4, 'positive_only': True, 'verbose': False}
2024-11-26 00:57:27,044 - INFO - Initializing recommender...
2024-11-26 00:57:27,045 - ERROR - Error in trial 0:
2024-11-26 00:57:27,053 - ERROR - Traceback (most recent call last):
  File "/var/folders/t_/9cbyldyd2bv5q2656cp65pbr0000gn/T/ipykernel_39320/27809788.py", line 44, in objective
    recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=urm_train)
NameError: name 'MultiThreadSLIM_SLIMElasticNetRecommender' is not defined

[I 2024-11-26 00:57:27,056] Trial 0 finished with value: -inf and parameters: {'alpha': 0.04216944928979198, 'l1_ratio': 0.874776361499749, 'topK'

EvaluatorHoldout: Ignoring 1909 ( 5.3%) Users that have less than 1 test interactions


2024-11-26 00:57:27,250 - INFO - Parameters: {'alpha': 0.1064262463259804, 'l1_ratio': 0.355799248807974, 'topK': 445, 'workers': 4, 'positive_only': True, 'verbose': False}
2024-11-26 00:57:27,251 - INFO - Initializing recommender...
2024-11-26 00:57:27,252 - ERROR - Error in trial 12:
2024-11-26 00:57:27,252 - ERROR - Traceback (most recent call last):
  File "/var/folders/t_/9cbyldyd2bv5q2656cp65pbr0000gn/T/ipykernel_39320/27809788.py", line 44, in objective
    recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=urm_train)
NameError: name 'MultiThreadSLIM_SLIMElasticNetRecommender' is not defined

[I 2024-11-26 00:57:27,253] Trial 12 finished with value: -inf and parameters: {'alpha': 0.1064262463259804, 'l1_ratio': 0.355799248807974, 'topK': 445}. Best is trial 0 with value: -inf.
2024-11-26 00:57:27,264 - INFO - 
Starting Trial 13
2024-11-26 00:57:27,265 - INFO - Parameters: {'alpha': 0.06254413600484016, 'l1_ratio': 0.08198631431816564, 'topK': 936, 'workers': 4, '

Best Hyperparameters:
{'alpha': 0.04216944928979198, 'l1_ratio': 0.874776361499749, 'topK': 218}
Best NDCG@10:
-inf


In [34]:
# Initialize with both URM and ICM
rp3beta_icm = RP3betaRecommenderICM(urm_train, icm_matrix)

# Fit with desired parameters
rp3beta_icm.fit(
    topK= 45,
    alpha= 1.8233706541466446,
    beta= 0.3532994315100997,
    delta= 0.22907169615669423,
    min_rating= 0.9940576605024676,
    implicit= True,
    normalize_similarity= True
)

RP3betaRecommenderICM: Normalized URM and ICM
RP3betaRecommenderICM: Processed 38121 ( 100.00% ) in 2.72 min. Items per second: 234
RP3betaRecommenderICM: Computation completed


In [35]:
# Initialize the evaluator with the test user-item matrix
evaluator = EvaluatorHoldout(urm_test, cutoff_list=[10] )

# Evaluate the TopPopularRecommender
results_df, results_run_string = evaluator.evaluateRecommender(rp3beta_icm)

# Print the evaluation results
print(results_run_string)

EvaluatorHoldout: Ignoring 175 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35561 (100.0%) in 19.65 sec. Users per second: 1810
CUTOFF: 10 - PRECISION: 0.0896122, PRECISION_RECALL_MIN_DEN: 0.1352885, RECALL: 0.1161388, MAP: 0.0443712, MAP_MIN_DEN: 0.0657579, MRR: 0.2526031, NDCG: 0.1292952, F1: 0.1011655, HIT_RATE: 0.5100813, ARHR_ALL_HITS: 0.3307186, NOVELTY: 0.0037006, AVERAGE_POPULARITY: 0.1146078, DIVERSITY_MEAN_INTER_LIST: 0.9965679, DIVERSITY_HERFINDAHL: 0.9996540, COVERAGE_ITEM: 0.5544975, COVERAGE_ITEM_HIT: 0.2335458, ITEMS_IN_GT: 0.9957504, COVERAGE_USER: 0.9951030, COVERAGE_USER_HIT: 0.5075834, USERS_IN_GT: 0.9951030, DIVERSITY_GINI: 0.1705873, SHANNON_ENTROPY: 12.8830481, RATIO_DIVERSITY_HERFINDAHL: 0.9997118, RATIO_DIVERSITY_GINI: 0.2811865, RATIO_SHANNON_ENTROPY: 0.8728449, RATIO_AVERAGE_POPULARITY: 1.5680361, RATIO_NOVELTY: 0.2765146, 



In [28]:
# Initialize with both URM and ICM
recommender = RP3betaRecommenderICM(urm_train, icm_matrix)

# Fit with desired parameters
recommender.fit(
    topK= 17,
    alpha= 0.9996386762753586,
    beta= 0.21406242351456914,
    delta= 0.17583769441037236,
    min_rating= 0.4181995527765843,
    implicit= False,
    normalize_similarity= True
)

RP3betaRecommenderICM: Normalized URM and ICM
RP3betaRecommenderICM: Processed 38121 ( 100.00% ) in 2.61 min. Items per second: 244
RP3betaRecommenderICM: Computation completed


In [29]:
# Initialize the evaluator with the test user-item matrix
evaluator = EvaluatorHoldout(urm_test, cutoff_list=[10] )

# Evaluate the TopPopularRecommender
results_df, results_run_string = evaluator.evaluateRecommender(recommender)

# Print the evaluation results
print(results_run_string)

EvaluatorHoldout: Ignoring 175 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35561 (100.0%) in 17.16 sec. Users per second: 2072
CUTOFF: 10 - PRECISION: 0.0863727, PRECISION_RECALL_MIN_DEN: 0.1306010, RECALL: 0.1120475, MAP: 0.0432113, MAP_MIN_DEN: 0.0643592, MRR: 0.2499103, NDCG: 0.1261464, F1: 0.0975490, HIT_RATE: 0.4984955, ARHR_ALL_HITS: 0.3246523, NOVELTY: 0.0037683, AVERAGE_POPULARITY: 0.0941379, DIVERSITY_MEAN_INTER_LIST: 0.9980544, DIVERSITY_HERFINDAHL: 0.9998026, COVERAGE_ITEM: 0.6986176, COVERAGE_ITEM_HIT: 0.2561580, ITEMS_IN_GT: 0.9957504, COVERAGE_USER: 0.9951030, COVERAGE_USER_HIT: 0.4960544, USERS_IN_GT: 0.9951030, DIVERSITY_GINI: 0.2455011, SHANNON_ENTROPY: 13.4618740, RATIO_DIVERSITY_HERFINDAHL: 0.9998605, RATIO_DIVERSITY_GINI: 0.4046702, RATIO_SHANNON_ENTROPY: 0.9120612, RATIO_AVERAGE_POPULARITY: 1.2879710, RATIO_NOVELTY: 0.2815729, 



In [31]:
# Initialize with both URM and ICM
recommender = RP3betaRecommenderICM(urm_train, icm_matrix)

# Fit with desired parameters
recommender.fit(
    topK= 17,
    alpha= 0.9996386762753586,
    beta= 0.21406242351456914,
    delta= 0.17583769441037236,
    min_rating= 0.4181995527765843,
    implicit= True,
    normalize_similarity= True
)

RP3betaRecommenderICM: Normalized URM and ICM
RP3betaRecommenderICM: Processed 38121 ( 100.00% ) in 2.60 min. Items per second: 245
RP3betaRecommenderICM: Computation completed


In [32]:
# Initialize the evaluator with the test user-item matrix
evaluator = EvaluatorHoldout(urm_test, cutoff_list=[10] )

# Evaluate the TopPopularRecommender
results_df, results_run_string = evaluator.evaluateRecommender(recommender)

# Print the evaluation results
print(results_run_string)

EvaluatorHoldout: Ignoring 175 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35561 (100.0%) in 16.92 sec. Users per second: 2102
CUTOFF: 10 - PRECISION: 0.0863727, PRECISION_RECALL_MIN_DEN: 0.1306010, RECALL: 0.1120475, MAP: 0.0432113, MAP_MIN_DEN: 0.0643592, MRR: 0.2499103, NDCG: 0.1261464, F1: 0.0975490, HIT_RATE: 0.4984955, ARHR_ALL_HITS: 0.3246523, NOVELTY: 0.0037683, AVERAGE_POPULARITY: 0.0941379, DIVERSITY_MEAN_INTER_LIST: 0.9980544, DIVERSITY_HERFINDAHL: 0.9998026, COVERAGE_ITEM: 0.6986176, COVERAGE_ITEM_HIT: 0.2561580, ITEMS_IN_GT: 0.9957504, COVERAGE_USER: 0.9951030, COVERAGE_USER_HIT: 0.4960544, USERS_IN_GT: 0.9951030, DIVERSITY_GINI: 0.2455011, SHANNON_ENTROPY: 13.4618740, RATIO_DIVERSITY_HERFINDAHL: 0.9998605, RATIO_DIVERSITY_GINI: 0.4046702, RATIO_SHANNON_ENTROPY: 0.9120612, RATIO_AVERAGE_POPULARITY: 1.2879710, RATIO_NOVELTY: 0.2815729, 



## ItemKNN

In [13]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

In [ ]:
knn_cf = ItemKNNCFRecommender(urm_train)
knn_cf.fit(
    topK=100,
    shrink=100,
    similarity="asymmetric",  # Use asymmetric similarity
    normalize=True,
    feature_weighting="BM25", # or "TF-IDF" or "none"
    asymmetric_alpha =0.3
)

Similarity column 38121 (100.0%), 2784.72 column/sec. Elapsed time 13.69 sec


In [18]:
# Initialize the evaluator with the test user-item matrix
evaluator = EvaluatorHoldout(urm_test, cutoff_list=[10] )

# Evaluate the TopPopularRecommender
results_df, results_run_string = evaluator.evaluateRecommender(knn_cf)

# Print the evaluation results
print(results_run_string)

EvaluatorHoldout: Ignoring 175 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35561 (100.0%) in 22.79 sec. Users per second: 1560
CUTOFF: 10 - PRECISION: 0.0796378, PRECISION_RECALL_MIN_DEN: 0.1209898, RECALL: 0.1040649, MAP: 0.0389336, MAP_MIN_DEN: 0.0581688, MRR: 0.2323352, NDCG: 0.1160464, F1: 0.0902273, HIT_RATE: 0.4792891, ARHR_ALL_HITS: 0.2976175, NOVELTY: 0.0038064, AVERAGE_POPULARITY: 0.0721470, DIVERSITY_MEAN_INTER_LIST: 0.9986316, DIVERSITY_HERFINDAHL: 0.9998604, COVERAGE_ITEM: 0.6769759, COVERAGE_ITEM_HIT: 0.2622177, ITEMS_IN_GT: 0.9957504, COVERAGE_USER: 0.9951030, COVERAGE_USER_HIT: 0.4769420, USERS_IN_GT: 0.9951030, DIVERSITY_GINI: 0.2462477, SHANNON_ENTROPY: 13.5626774, RATIO_DIVERSITY_HERFINDAHL: 0.9999182, RATIO_DIVERSITY_GINI: 0.4059009, RATIO_SHANNON_ENTROPY: 0.9188908, RATIO_AVERAGE_POPULARITY: 0.9870973, RATIO_NOVELTY: 0.2844203, 



In [41]:
import optuna
import logging
import os
from datetime import datetime
from Evaluation.Evaluator import EvaluatorHoldout

def optimize_item_knn_cf(urm_train, urm_validation, n_trials=30, output_folder="optimization_results"):
    # Setup logging
    os.makedirs(output_folder, exist_ok=True)
    log_file = os.path.join(output_folder, f'optimization_log_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')
    
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    
    evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])

    def objective(trial):
        # Define hyperparameter ranges
        params = {
            "topK": trial.suggest_int("topK", 10, 200),
            "shrink": trial.suggest_int("shrink", 0, 300),
            "similarity": trial.suggest_categorical("similarity", ["cosine", "asymmetric", "jaccard", "tanimoto"]),
            "normalize": trial.suggest_categorical("normalize", [True, False]),
            "feature_weighting": trial.suggest_categorical("feature_weighting", ["BM25", "TF-IDF", "none"]),
            "asymmetric_alpha": trial.suggest_float("asymmetric_alpha", 0.1, 2.0),
        }
        
        try:
            # Initialize and fit the recommender
            recommender = ItemKNNCFRecommender(URM_train=urm_train)
            recommender.fit(**params)
            
            # Evaluate the recommender
            results_df, _ = evaluator_validation.evaluateRecommender(recommender)
            map_at_10 = results_df.loc[10]["MAP"]
            
            logging.info(f"Trial {trial.number} results:")
            logging.info(f"Params: {params}")
            logging.info(f"MAP@10: {map_at_10}")
            
            return map_at_10
            
        except Exception as e:
            logging.error(f"Error in trial {trial.number}: {str(e)}")
            raise optuna.exceptions.TrialPruned()
    
    # Create the Optuna study
    study = optuna.create_study(direction="maximize")

    try:
        study.optimize(objective, n_trials=n_trials)
        
        # Log the best results
        logging.info("Optimization completed!")
        logging.info(f"Best trial: {study.best_trial.number}")
        logging.info(f"Best MAP@10: {study.best_value}")
        logging.info(f"Best hyperparameters: {study.best_params}")
        
        return study.best_params, study.best_value

    except Exception as e:
        logging.error(f"Optimization failed: {str(e)}")
        raise

In [43]:
if __name__ == "__main__":
    # Assuming urm_train and urm_validation are already defined
    best_params, best_map = optimize_item_knn_cf(
        urm_train=urm_train,
        urm_validation=urm_validation,
        n_trials=100
    )
    print(f"Best MAP@10: {best_map}")
    print(f"Best hyperparameters: {best_params}")


[I 2024-11-25 05:25:38,665] A new study created in memory with name: no-name-e7cf1e87-8b31-416e-9675-fa7734b89c3f


EvaluatorHoldout: Ignoring 1909 ( 5.3%) Users that have less than 1 test interactions
Similarity column 38121 (100.0%), 2870.11 column/sec. Elapsed time 13.28 sec
EvaluatorHoldout: Processed 33827 (100.0%) in 23.34 sec. Users per second: 1449


2024-11-25 05:26:15,843 - INFO - Trial 0 results:
2024-11-25 05:26:15,845 - INFO - Params: {'topK': 51, 'shrink': 198, 'similarity': 'asymmetric', 'normalize': False, 'feature_weighting': 'TF-IDF', 'asymmetric_alpha': 0.6940616774342656}
2024-11-25 05:26:15,846 - INFO - MAP@10: 0.014253392964482152
[I 2024-11-25 05:26:15,850] Trial 0 finished with value: 0.014253392964482152 and parameters: {'topK': 51, 'shrink': 198, 'similarity': 'asymmetric', 'normalize': False, 'feature_weighting': 'TF-IDF', 'asymmetric_alpha': 0.6940616774342656}. Best is trial 0 with value: 0.014253392964482152.


Similarity column 38121 (100.0%), 2759.90 column/sec. Elapsed time 13.81 sec
EvaluatorHoldout: Processed 33827 (100.0%) in 21.02 sec. Users per second: 1609


2024-11-25 05:26:50,940 - INFO - Trial 1 results:
2024-11-25 05:26:50,941 - INFO - Params: {'topK': 44, 'shrink': 76, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'none', 'asymmetric_alpha': 1.42348023227323}
2024-11-25 05:26:50,941 - INFO - MAP@10: 0.018029600661817684
[I 2024-11-25 05:26:50,945] Trial 1 finished with value: 0.018029600661817684 and parameters: {'topK': 44, 'shrink': 76, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'none', 'asymmetric_alpha': 1.42348023227323}. Best is trial 1 with value: 0.018029600661817684.


...

Similarity column 38121 (100.0%), 2930.19 column/sec. Elapsed time 13.01 sec
EvaluatorHoldout: Processed 33827 (100.0%) in 18.70 sec. Users per second: 1809


2024-11-25 06:19:20,329 - INFO - Trial 98 results:
2024-11-25 06:19:20,330 - INFO - Params: {'topK': 24, 'shrink': 270, 'similarity': 'jaccard', 'normalize': False, 'feature_weighting': 'none', 'asymmetric_alpha': 1.0263870985149475}
2024-11-25 06:19:20,331 - INFO - MAP@10: 0.01766256385783706
[I 2024-11-25 06:19:20,334] Trial 98 finished with value: 0.01766256385783706 and parameters: {'topK': 24, 'shrink': 270, 'similarity': 'jaccard', 'normalize': False, 'feature_weighting': 'none', 'asymmetric_alpha': 1.0263870985149475}. Best is trial 85 with value: 0.02015179829581033.


Similarity column 38121 (100.0%), 2915.05 column/sec. Elapsed time 13.08 sec
EvaluatorHoldout: Processed 33827 (100.0%) in 17.71 sec. Users per second: 1910


2024-11-25 06:19:51,336 - INFO - Trial 99 results:
2024-11-25 06:19:51,337 - INFO - Params: {'topK': 10, 'shrink': 299, 'similarity': 'jaccard', 'normalize': False, 'feature_weighting': 'BM25', 'asymmetric_alpha': 0.9756384422307992}
2024-11-25 06:19:51,338 - INFO - MAP@10: 0.020131989286288905
[I 2024-11-25 06:19:51,341] Trial 99 finished with value: 0.020131989286288905 and parameters: {'topK': 10, 'shrink': 299, 'similarity': 'jaccard', 'normalize': False, 'feature_weighting': 'BM25', 'asymmetric_alpha': 0.9756384422307992}. Best is trial 85 with value: 0.02015179829581033.
2024-11-25 06:19:51,342 - INFO - Optimization completed!
2024-11-25 06:19:51,342 - INFO - Best trial: 85
2024-11-25 06:19:51,343 - INFO - Best MAP@10: 0.02015179829581033
2024-11-25 06:19:51,345 - INFO - Best hyperparameters: {'topK': 10, 'shrink': 264, 'similarity': 'jaccard', 'normalize': False, 'feature_weighting': 'BM25', 'asymmetric_alpha': 1.2043488778583}


Best MAP@10: 0.02015179829581033
Best hyperparameters: {'topK': 10, 'shrink': 264, 'similarity': 'jaccard', 'normalize': False, 'feature_weighting': 'BM25', 'asymmetric_alpha': 1.2043488778583}


In [44]:
#insert opt parameters
knn_cf = ItemKNNCFRecommender(urm_train)
knn_cf.fit(
    topK=10,
    shrink=264,
    similarity="jaccard",  
    normalize=False,
    feature_weighting="BM25", # or "TF-IDF" or "none"
    asymmetric_alpha =1.2043488778583
)

Similarity column 38121 (100.0%), 2878.58 column/sec. Elapsed time 13.24 sec


In [45]:
from Recommenders.GabsHybridTesting.HybridRP3KNN import HybridRP3betaKNNRecommender

hybrid_recommender = HybridRP3betaKNNRecommender(
    URM_train=urm_train,
    rp3beta_icm_recommender=rp3beta_icm,
    knn_recommender=knn_cf,
    verbose=True
)

HybridRP3betaKNNRecommender: Initialized


In [53]:
def objective(trial):
    # Define the hyperparameter search space
    params = {
        "alpha": trial.suggest_float("alpha", 0.0, 1.0),  # Expanded range
    }

    try:
        # Initialize the recommender
        recommender = HybridRP3betaKNNRecommender(URM_train=urm_train,  rp3beta_icm_recommender=rp3beta_icm, knn_recommender=knn_cf, verbose=True)
        recommender.fit(**params)  # Pass the params as keyword arguments

        # Evaluate the recommender
        evaluator = EvaluatorHoldout(urm_validation, cutoff_list=[10])
        results_df, _ = evaluator.evaluateRecommender(recommender)
        map_at_10 = results_df.loc[10]["MAP"]

        return map_at_10  # Objective to maximize

    except Exception as e:
        # Handle exceptions and prune trial if necessary
        raise optuna.exceptions.TrialPruned(f"Error in trial {trial.number}: {str(e)}")


In [54]:
import optuna

# Create the study
study = optuna.create_study(direction="maximize")  # Maximize MAP@10

# Optimize the objective function
study.optimize(objective, n_trials=50)  # Adjust n_trials as needed

# Print the best results
print(f"Best MAP@10: {study.best_value}")
print(f"Best hyperparameters: {study.best_params}")

[I 2024-11-25 14:44:40,742] A new study created in memory with name: no-name-c0886aff-7f7a-4ea6-8646-5be309215506


HybridRP3betaKNNRecommender: Initialized
HybridRP3betaKNNRecommender: Combining matrices with alpha=0.6518977791009515
HybridRP3betaKNNRecommender: Computation completed
EvaluatorHoldout: Ignoring 1909 ( 5.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 33827 (100.0%) in 18.37 sec. Users per second: 1841


[I 2024-11-25 14:44:59,224] Trial 0 finished with value: 0.020584458455980965 and parameters: {'alpha': 0.6518977791009515}. Best is trial 0 with value: 0.020584458455980965.


HybridRP3betaKNNRecommender: Initialized
HybridRP3betaKNNRecommender: Combining matrices with alpha=0.09240038153090468
HybridRP3betaKNNRecommender: Computation completed
EvaluatorHoldout: Ignoring 1909 ( 5.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 33827 (100.0%) in 18.06 sec. Users per second: 1873


[I 2024-11-25 14:45:17,359] Trial 1 finished with value: 0.020099380554933613 and parameters: {'alpha': 0.09240038153090468}. Best is trial 0 with value: 0.020584458455980965.


...

HybridRP3betaKNNRecommender: Initialized
HybridRP3betaKNNRecommender: Combining matrices with alpha=0.21861356542814225
HybridRP3betaKNNRecommender: Computation completed
EvaluatorHoldout: Ignoring 1909 ( 5.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 33827 (100.0%) in 18.00 sec. Users per second: 1879


[I 2024-11-25 14:59:30,077] Trial 48 finished with value: 0.021024778975749878 and parameters: {'alpha': 0.21861356542814225}. Best is trial 32 with value: 0.02109934958502648.


HybridRP3betaKNNRecommender: Initialized
HybridRP3betaKNNRecommender: Combining matrices with alpha=0.5224204250710761
HybridRP3betaKNNRecommender: Computation completed
EvaluatorHoldout: Ignoring 1909 ( 5.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 33827 (100.0%) in 17.97 sec. Users per second: 1882


[I 2024-11-25 14:59:48,136] Trial 49 finished with value: 0.020824671144164083 and parameters: {'alpha': 0.5224204250710761}. Best is trial 32 with value: 0.02109934958502648.


Best MAP@10: 0.02109934958502648
Best hyperparameters: {'alpha': 0.3053446904625561}


In [ ]:
best_params = study.best_params  # Retrieve best parameters
print(f"Best hyperparameters: {best_params}")  # Print the best hyperparameters

In [56]:
hybrid_recommender.fit(alpha=0.3053446904625561, normalize_similarity=True)

HybridRP3betaKNNRecommender: Combining matrices with alpha=0.3053446904625561
HybridRP3betaKNNRecommender: Computation completed


In [57]:
# Initialize the evaluator with the test user-item matrix
evaluator = EvaluatorHoldout(urm_test, cutoff_list=[10] )

# Evaluate the TopPopularRecommender
results_df, results_run_string = evaluator.evaluateRecommender(hybrid_recommender)

# Print the evaluation results
print(results_run_string)

EvaluatorHoldout: Ignoring 175 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35561 (100.0%) in 18.87 sec. Users per second: 1884
CUTOFF: 10 - PRECISION: 0.0915188, PRECISION_RECALL_MIN_DEN: 0.1384221, RECALL: 0.1189492, MAP: 0.0452405, MAP_MIN_DEN: 0.0668279, MRR: 0.2542777, NDCG: 0.1314160, F1: 0.1034465, HIT_RATE: 0.5166615, ARHR_ALL_HITS: 0.3351236, NOVELTY: 0.0037665, AVERAGE_POPULARITY: 0.0917412, DIVERSITY_MEAN_INTER_LIST: 0.9979220, DIVERSITY_HERFINDAHL: 0.9997894, COVERAGE_ITEM: 0.6154875, COVERAGE_ITEM_HIT: 0.2674117, ITEMS_IN_GT: 0.9957504, COVERAGE_USER: 0.9951030, COVERAGE_USER_HIT: 0.5141314, USERS_IN_GT: 0.9951030, DIVERSITY_GINI: 0.2214044, SHANNON_ENTROPY: 13.3383541, RATIO_DIVERSITY_HERFINDAHL: 0.9998472, RATIO_DIVERSITY_GINI: 0.3649506, RATIO_SHANNON_ENTROPY: 0.9036926, RATIO_AVERAGE_POPULARITY: 1.2551798, RATIO_NOVELTY: 0.2814395, 



# SLIM ElasticNet




In [59]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

# Initialize the SLIM ElasticNet recommender
slim_recommender = SLIMElasticNetRecommender(urm_train)
slim_recommender.fit()

SLIMElasticNetRecommender: Processed 34036 (89.3%) in 5.00 min. Items per second: 113.45
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 5.56 min. Items per second: 114.23


In [60]:
# Initialize the evaluator with the test user-item matrix
evaluator = EvaluatorHoldout(urm_test, cutoff_list=[10] )

# Evaluate the TopPopularRecommender
results_df, results_run_string = evaluator.evaluateRecommender(slim_recommender)

# Print the evaluation results
print(results_run_string)

EvaluatorHoldout: Ignoring 175 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35561 (100.0%) in 16.94 sec. Users per second: 2100
CUTOFF: 10 - PRECISION: 0.0008689, PRECISION_RECALL_MIN_DEN: 0.0012502, RECALL: 0.0010216, MAP: 0.0002834, MAP_MIN_DEN: 0.0004047, MRR: 0.0026065, NDCG: 0.0010940, F1: 0.0009391, HIT_RATE: 0.0081269, ARHR_ALL_HITS: 0.0027140, NOVELTY: 0.0035503, AVERAGE_POPULARITY: 0.1787349, DIVERSITY_MEAN_INTER_LIST: 0.5258451, DIVERSITY_HERFINDAHL: 0.9525830, COVERAGE_ITEM: 0.0062433, COVERAGE_ITEM_HIT: 0.0011280, ITEMS_IN_GT: 0.9957504, COVERAGE_USER: 0.9951030, COVERAGE_USER_HIT: 0.0080871, USERS_IN_GT: 0.9951030, DIVERSITY_GINI: 0.0005903, SHANNON_ENTROPY: 4.8382483, RATIO_DIVERSITY_HERFINDAHL: 0.9526382, RATIO_DIVERSITY_GINI: 0.0009730, RATIO_SHANNON_ENTROPY: 0.3277982, RATIO_AVERAGE_POPULARITY: 2.4454065, RATIO_NOVELTY: 0.2652854, 



In [69]:
import optuna
from Evaluation.Evaluator import EvaluatorHoldout

# Define the number of trials
n_trials = 30

# Define the evaluator for validation
evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])

# Define the objective function
def objective(trial):
    # Define hyperparameters to optimize
    l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0)  # ElasticNet ratio
    alpha = trial.suggest_float("alpha", 1e-4, 1e1, log=True)  # Regularization strength
    positive_only = trial.suggest_categorical("positive_only", [True, False])
    topK = trial.suggest_int("topK", 10, 200)  # TopK for similarity matrix
    
    # Initialize and fit the SLIM ElasticNet Recommender
    recommender = SLIMElasticNetRecommender(URM_train=urm_train)
    recommender.fit(l1_ratio=l1_ratio, alpha=alpha, positive_only=positive_only, topK=topK)
    
    # Evaluate using the validation set
    results_df, _ = evaluator_validation.evaluateRecommender(recommender)
    map_at_10 = results_df.loc[10]["MAP"]  # Extract MAP@10 from results
    
    return map_at_10

EvaluatorHoldout: Ignoring 1909 ( 5.3%) Users that have less than 1 test interactions


In [70]:
# Create and run the Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=n_trials)

# Output the best hyperparameters found
print("Best hyperparameters:", study.best_params)
print("Best MAP@10:", study.best_value)

[I 2024-11-26 01:00:52,546] A new study created in memory with name: no-name-2b4e1c5b-0629-4b6a-a318-23b3d05f0b63


SLIMElasticNetRecommender: Processed 30745 (80.7%) in 5.00 min. Items per second: 102.48
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 6.14 min. Items per second: 103.45
EvaluatorHoldout: Processed 33827 (100.0%) in 18.87 sec. Users per second: 1793


[I 2024-11-26 01:07:20,215] Trial 0 finished with value: 0.009306577914420907 and parameters: {'l1_ratio': 0.5286860498510946, 'alpha': 0.0016035036587517737, 'positive_only': True, 'topK': 42}. Best is trial 0 with value: 0.009306577914420907.


SLIMElasticNetRecommender: Processed 7049 (18.5%) in 5.00 min. Items per second: 23.49
SLIMElasticNetRecommender: Processed 13933 (36.5%) in 10.00 min. Items per second: 23.22
SLIMElasticNetRecommender: Processed 21318 (55.9%) in 15.00 min. Items per second: 23.68
SLIMElasticNetRecommender: Processed 28199 (74.0%) in 20.00 min. Items per second: 23.50
SLIMElasticNetRecommender: Processed 35854 (94.1%) in 25.00 min. Items per second: 23.90
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 26.61 min. Items per second: 23.88
EvaluatorHoldout: Processed 33827 (100.0%) in 25.51 sec. Users per second: 1326


[I 2024-11-26 01:34:22,365] Trial 1 finished with value: 0.02232020913133592 and parameters: {'l1_ratio': 0.4416370900210117, 'alpha': 0.00018443701158134376, 'positive_only': False, 'topK': 109}. Best is trial 1 with value: 0.02232020913133592.


...

SLIMElasticNetRecommender: Processed 38105 (100.0%) in 5.00 min. Items per second: 127.01
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 5.00 min. Items per second: 127.01
EvaluatorHoldout: Processed 33827 (100.0%) in 14.66 sec. Users per second: 2307


[I 2024-11-26 07:46:54,678] Trial 28 finished with value: 0.005921258541946145 and parameters: {'l1_ratio': 0.3744052711677878, 'alpha': 0.0038395479766108924, 'positive_only': False, 'topK': 89}. Best is trial 20 with value: 0.022474772429837506.


SLIMElasticNetRecommender: Processed 14560 (38.2%) in 5.00 min. Items per second: 48.53
SLIMElasticNetRecommender: Processed 29991 (78.7%) in 10.00 min. Items per second: 49.98
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 12.40 min. Items per second: 51.22
EvaluatorHoldout: Processed 33827 (100.0%) in 16.22 sec. Users per second: 2086


[I 2024-11-26 07:59:35,181] Trial 29 finished with value: 0.020305865371936713 and parameters: {'l1_ratio': 0.4810000935834188, 'alpha': 0.00034537261477321935, 'positive_only': True, 'topK': 42}. Best is trial 20 with value: 0.022474772429837506.


Best hyperparameters: {'l1_ratio': 0.4408355927953408, 'alpha': 0.00013519978876092592, 'positive_only': False, 'topK': 59}
Best MAP@10: 0.022474772429837506


In [71]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

# Initialize the SLIM ElasticNet recommender
slim_recommender = SLIMElasticNetRecommender(urm_train)
slim_recommender.fit(l1_ratio= 0.4408355927953408, alpha= 0.00013519978876092592, positive_only= False, topK= 59)

SLIMElasticNetRecommender: Processed 6424 (16.9%) in 5.00 min. Items per second: 21.41
SLIMElasticNetRecommender: Processed 12916 (33.9%) in 10.00 min. Items per second: 21.52
SLIMElasticNetRecommender: Processed 20566 (53.9%) in 15.00 min. Items per second: 22.85
SLIMElasticNetRecommender: Processed 28265 (74.1%) in 20.00 min. Items per second: 23.55
SLIMElasticNetRecommender: Processed 36363 (95.4%) in 25.00 min. Items per second: 24.24
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 26.19 min. Items per second: 24.26


In [72]:
# Initialize the evaluator with the test user-item matrix
evaluator = EvaluatorHoldout(urm_test, cutoff_list=[10] )

# Evaluate the TopPopularRecommender
results_df, results_run_string = evaluator.evaluateRecommender(slim_recommender)

# Print the evaluation results
print(results_run_string)

EvaluatorHoldout: Ignoring 175 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35561 (100.0%) in 23.47 sec. Users per second: 1515
CUTOFF: 10 - PRECISION: 0.0952392, PRECISION_RECALL_MIN_DEN: 0.1430107, RECALL: 0.1221699, MAP: 0.0483441, MAP_MIN_DEN: 0.0712179, MRR: 0.2698329, NDCG: 0.1381397, F1: 0.1070366, HIT_RATE: 0.5299064, ARHR_ALL_HITS: 0.3563031, NOVELTY: 0.0037323, AVERAGE_POPULARITY: 0.0950999, DIVERSITY_MEAN_INTER_LIST: 0.9983186, DIVERSITY_HERFINDAHL: 0.9998291, COVERAGE_ITEM: 0.5911702, COVERAGE_ITEM_HIT: 0.2734713, ITEMS_IN_GT: 0.9957504, COVERAGE_USER: 0.9951030, COVERAGE_USER_HIT: 0.5273114, USERS_IN_GT: 0.9951030, DIVERSITY_GINI: 0.2311326, SHANNON_ENTROPY: 13.4244003, RATIO_DIVERSITY_HERFINDAHL: 0.9998869, RATIO_DIVERSITY_GINI: 0.3809860, RATIO_SHANNON_ENTROPY: 0.9095223, RATIO_AVERAGE_POPULARITY: 1.3011331, RATIO_NOVELTY: 0.2788784, 



In [9]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

# Initialize the SLIM ElasticNet recommender
slim_recommender = SLIMElasticNetRecommender(urm_all)
slim_recommender.fit(l1_ratio= 0.4408355927953408, alpha= 0.00013519978876092592, positive_only= False, topK= 59)

SLIMElasticNetRecommender: Processed 3625 ( 9.5%) in 5.00 min. Items per second: 12.08
SLIMElasticNetRecommender: Processed 7292 (19.1%) in 10.00 min. Items per second: 12.15
SLIMElasticNetRecommender: Processed 11025 (28.9%) in 15.00 min. Items per second: 12.25
SLIMElasticNetRecommender: Processed 14858 (39.0%) in 20.00 min. Items per second: 12.38
SLIMElasticNetRecommender: Processed 18410 (48.3%) in 25.01 min. Items per second: 12.27
SLIMElasticNetRecommender: Processed 22110 (58.0%) in 30.03 min. Items per second: 12.27
SLIMElasticNetRecommender: Processed 25675 (67.4%) in 35.03 min. Items per second: 12.21
SLIMElasticNetRecommender: Processed 28729 (75.4%) in 40.04 min. Items per second: 11.96
SLIMElasticNetRecommender: Processed 32496 (85.2%) in 45.04 min. Items per second: 12.03
SLIMElasticNetRecommender: Processed 36130 (94.8%) in 50.04 min. Items per second: 12.03
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 52.85 min. Items per second: 12.02


# MultVAE

In [11]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender

# Initialize the MultVAE Recommender
vae_recommender = MultVAERecommender(URM_train=urm_train)

# Fit the model with appropriate hyperparameters
vae_recommender.fit(
    epochs=100, 
    learning_rate=1e-3, 
    batch_size=500, 
    dropout=0.5, 
    total_anneal_steps=200000, 
    anneal_cap=0.2, 
    p_dims=[200, 600], 
    l2_reg=0.01
)

MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_30/'
MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_30/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Epoch 1 of 100. Elapsed time 17.00 sec
MultVAERecommender: Epoch 2 of 100. Elapsed time 31.84 sec
MultVAERecommender: Epoch 3 of 100. Elapsed time 46.82 sec
MultVAERecommender: Epoch 4 of 100. Elapsed time 1.03 min
MultVAERecommender: Epoch 5 of 100. Elapsed time 1.28 min
MultVAERecommender: Cleaning temporary files from './result_experiments/__Temp_MultVAERecommender_30/'


KeyboardInterrupt: 

In [11]:
import optuna
from Evaluation.Evaluator import EvaluatorHoldout

# Define the number of trials
n_trials = 30

# Define the evaluator for validation
evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])


EvaluatorHoldout: Ignoring 1909 ( 5.3%) Users that have less than 1 test interactions


In [14]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
# Define the objective function
def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [128, 256, 500, 1024])
    dropout = trial.suggest_float("dropout", 0.1, 0.9)
    total_anneal_steps = trial.suggest_int("total_anneal_steps", 10000, 500000)
    anneal_cap = trial.suggest_float("anneal_cap", 0.1, 0.5)
    l2_reg = trial.suggest_float("l2_reg", 1e-5, 1e-2, log=True)
    
    # Use tuples for p_dims to avoid warnings
    p_dims_tuple = trial.suggest_categorical("p_dims", [(200, 600), (300, 600, 800), (128, 256, 512)])
    p_dims = list(p_dims_tuple)  # Convert the tuple back to a list

    # Initialize the recommender
    vae_recommender = MultVAERecommender(URM_train=urm_train)
    
    # Fit the model with the suggested hyperparameters
    vae_recommender.fit(
        epochs=10,
        learning_rate=learning_rate,
        batch_size=batch_size,
        dropout=dropout,
        total_anneal_steps=total_anneal_steps,
        anneal_cap=anneal_cap,
        p_dims=p_dims,
        l2_reg=l2_reg,
        validation_every_n=5,
        stop_on_validation=True,
        validation_metric="MAP",
        lower_validations_allowed=3
    )
    
    # Use EvaluatorHoldout to calculate MAP@10
    results_df, _ = evaluator_validation.evaluateRecommender(vae_recommender)
    map_at_10 = results_df.loc[10]["MAP"]  # Extract MAP@10 from results
    
    return map_at_10


In [15]:
# Create and run the Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=n_trials)

# Output the best hyperparameters found
print("Best hyperparameters:", study.best_params)
print("Best MAP@10:", study.best_value)


[I 2024-11-20 10:22:45,328] A new study created in memory with name: no-name-4180292c-0a45-497b-9fbb-4f7388ba508e
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (200, 600) which is of type tuple.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (300, 600, 800) which is of type tuple.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (128, 256, 512) which is of type tuple.
  warnings.warn(message)


MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_30/'
MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_30/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Epoch 1 of 10. Elapsed time 10.72 sec
MultVAERecommender: Epoch 2 of 10. Elapsed time 20.00 sec
MultVAERecommender: Epoch 3 of 10. Elapsed time 29.36 sec
MultVAERecommender: Epoch 4 of 10. Elapsed time 38.79 sec
MultVAERecommender: Epoch 5 of 10. Elapsed time 48.19 sec
MultVAERecommender: Epoch 6 of 10. Elapsed time 57.54 sec
MultVAERecommender: Epoch 7 of 10. Elapsed time 1.11 min
MultVAERecommender: Epoch 8 of 10. Elapsed time 1.27 min
MultVAERecommender: Epoch 9 of 10. Elapsed time 1.43 min
MultVAERecommender: Epoch 10 of 10. Elapsed time 1.58 min
MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_30/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Terminating at epoch 1

[I 2024-11-20 10:25:10,597] Trial 0 finished with value: 0.0053837523420992295 and parameters: {'learning_rate': 0.0006351513277457071, 'batch_size': 128, 'dropout': 0.7368000927719138, 'total_anneal_steps': 349611, 'anneal_cap': 0.47994633137437326, 'l2_reg': 0.0033438675855060208, 'p_dims': (300, 600, 800)}. Best is trial 0 with value: 0.0053837523420992295.


MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_30/'


/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (200, 600) which is of type tuple.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (300, 600, 800) which is of type tuple.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (128, 256, 512) which is of type tuple.
  warnings.warn(message)


...

MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_30/'


/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (200, 600) which is of type tuple.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (300, 600, 800) which is of type tuple.
  warnings.warn(message)
/opt/conda/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (128, 256, 512) which is of type tuple.
  warnings.warn(message)


MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_30/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Epoch 1 of 10. Elapsed time 9.58 sec
MultVAERecommender: Epoch 2 of 10. Elapsed time 17.80 sec
MultVAERecommender: Epoch 3 of 10. Elapsed time 25.99 sec
MultVAERecommender: Epoch 4 of 10. Elapsed time 34.19 sec
MultVAERecommender: Epoch 5 of 10. Elapsed time 42.38 sec
MultVAERecommender: Epoch 6 of 10. Elapsed time 50.63 sec
MultVAERecommender: Epoch 7 of 10. Elapsed time 58.83 sec
MultVAERecommender: Epoch 8 of 10. Elapsed time 1.12 min
MultVAERecommender: Epoch 9 of 10. Elapsed time 1.25 min
MultVAERecommender: Epoch 10 of 10. Elapsed time 1.38 min
MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_30/_best_model'
MultVAERecommender: Saving complete
MultVAERecommender: Terminating at epoch 10. Elapsed time 1.40 min
MultVAERecommender: Loading model from file './result_experiments/__Temp_M

[I 2024-11-20 11:58:25,838] Trial 29 finished with value: 0.008311403354416427 and parameters: {'learning_rate': 0.0005749088074405894, 'batch_size': 128, 'dropout': 0.5171899626684193, 'total_anneal_steps': 56324, 'anneal_cap': 0.28491643271443967, 'l2_reg': 3.3722284865128834e-05, 'p_dims': (128, 256, 512)}. Best is trial 25 with value: 0.010276267994806656.


Best hyperparameters: {'learning_rate': 0.0013739782600272086, 'batch_size': 256, 'dropout': 0.1700020997233899, 'total_anneal_steps': 64697, 'anneal_cap': 0.27633313082113736, 'l2_reg': 1.0319354996369627e-05, 'p_dims': (200, 600)}
Best MAP@10: 0.010276267994806656


In [33]:
best_params = {
    "learning_rate": 0.0013739782600272086,
    "batch_size": 256,
    "dropout": 0.1700020997233899,
    "total_anneal_steps": 64697,
    "anneal_cap": 0.27633313082113736,
    "l2_reg": 1.0319354996369627e-05,
    "p_dims": [200, 600],  # Converted from tuple to list
}


In [ ]:
# Retrieve the best hyperparameters from the Optuna study
best_params = study.best_params

# Convert p_dims back from a tuple if needed
best_params["p_dims"] = list(best_params["p_dims"])

KeyError: 'p_dims'

In [ ]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender

# Initialize the MultVAE Recommender with the optimal hyperparameters
vae_recommender = MultVAERecommender(URM_train=urm_train)

# Fit the model with the optimal hyperparameters
vae_recommender.fit(
    epochs=100,  # Full training after finding the best hyperparameters
    learning_rate=best_params["learning_rate"],
    batch_size=best_params["batch_size"],
    dropout=best_params["dropout"],
    total_anneal_steps=best_params["total_anneal_steps"],
    anneal_cap=best_params["anneal_cap"],
    p_dims=best_params["p_dims"],
    l2_reg=best_params["l2_reg"],
    validation_every_n=5,
    stop_on_validation=True,
    validation_metric="MAP",
    lower_validations_allowed=3
)

MultVAERecommender: Using default Temp folder './result_experiments/__Temp_MultVAERecommender_30/'
MultVAERecommender: Saving model in file './result_experiments/__Temp_MultVAERecommender_30/_best_model'
MultVAERecommender: Saving complete


In [17]:
# Initialize the evaluator with the test user-item matrix
evaluator = EvaluatorHoldout(urm_test, cutoff_list=[10])

# Evaluate the MultVAERecommender
results_df, results_run_string = evaluator.evaluateRecommender(vae_recommender)

# Print the evaluation results
print(results_run_string)

EvaluatorHoldout: Ignoring 175 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35561 (100.0%) in 42.53 sec. Users per second: 836
CUTOFF: 10 - PRECISION: 0.0515368, PRECISION_RECALL_MIN_DEN: 0.0794196, RECALL: 0.0692948, MAP: 0.0218639, MAP_MIN_DEN: 0.0336291, MRR: 0.1527365, NDCG: 0.0731797, F1: 0.0591109, HIT_RATE: 0.3646410, ARHR_ALL_HITS: 0.1815375, NOVELTY: 0.0035525, AVERAGE_POPULARITY: 0.1689908, DIVERSITY_MEAN_INTER_LIST: 0.9907014, DIVERSITY_HERFINDAHL: 0.9990674, COVERAGE_ITEM: 0.5319640, COVERAGE_ITEM_HIT: 0.1609874, ITEMS_IN_GT: 0.9957504, COVERAGE_USER: 0.9951030, COVERAGE_USER_HIT: 0.3628554, USERS_IN_GT: 0.9951030, DIVERSITY_GINI: 0.1191579, SHANNON_ENTROPY: 12.0617342, RATIO_DIVERSITY_HERFINDAHL: 0.9991252, RATIO_DIVERSITY_GINI: 0.1964132, RATIO_SHANNON_ENTROPY: 0.8171998, RATIO_AVERAGE_POPULARITY: 2.3120898, RATIO_NOVELTY: 0.2654441, 



In [ ]:
from lightfm import LightFM

# Initialize LightFM model
model = LightFM(loss='warp', no_components=30)

# Training loop with epoch progress tracking
num_epochs = 20
for epoch in range(1, num_epochs + 1):
    model.fit_partial(
        URM_train,
        item_features=icm_matrix,
        num_threads=4  # Multithreading for faster training
    )
    print(f"Epoch {epoch}/{num_epochs} completed")


In [ ]:
import numpy as np

def lightfm_recommend_batch(model, user_ids, URM_train, item_features, at=10):
    """
    Generate top-N recommendations for a batch of users using LightFM.
    
    Args:
        model: Trained LightFM model.
        user_ids: List of user IDs for which to generate recommendations.
        URM_train: Training matrix to exclude seen items.
        item_features: Optional LightFM item feature matrix.
        at: Number of recommendations to generate.

    Returns:
        List of lists containing recommended item IDs for each user.
    """
    num_items = URM_train.shape[1]
    recommendations = []
    
    for user_id in user_ids:
        scores = model.predict(user_id, np.arange(num_items), item_features=item_features)
        
        # Exclude items seen in training
        seen_items = URM_train[user_id].indices
        scores[seen_items] = -np.inf
        
        # Get top-N recommendations
        top_items = np.argsort(-scores)[:at]
        recommendations.append(top_items)
    
    return recommendations


In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout

# Initialize evaluator for validation
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

# Define a wrapper for the evaluator to use LightFM predictions
class LightFMRecommenderWrapper:
    def __init__(self, model, URM_train, item_features):
        self.model = model
        self.URM_train = URM_train
        self.item_features = item_features

    def recommend(self, user_id, at=10):
        scores = self.model.predict(user_id, np.arange(self.URM_train.shape[1]), item_features=self.item_features)
        seen_items = self.URM_train[user_id].indices
        scores[seen_items] = -np.inf
        return np.argsort(-scores)[:at]

# Wrap the LightFM model
lightfm_wrapper = LightFMRecommenderWrapper(model, URM_train, icm_matrix)

# Evaluate the wrapped model
results = evaluator_validation.evaluateRecommender(lightfm_wrapper)
print("Validation Results:", results)


# LightFM

In [24]:
import os
print(os.cpu_count())

4


In [13]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=464212 sha256=2047f8be4b04a1c571bae055a42257ed99ce24667ec0bf1050d65fd4ab1c7308
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [26]:
from lightfm.evaluation import precision_at_k, auc_score
import numpy as np

# Initialize the model
model = LightFM(loss='warp', no_components=30)

# Training loop with performance tracking
num_epochs = 20
for epoch in range(1, num_epochs + 1):
    # Partial fit for one epoch
    model.fit_partial(
        urm_train,
        item_features=icm_matrix,
        num_threads=4
    )
    
    # Evaluate the model
    precision = precision_at_k(model, urm_validation, item_features=icm_matrix, k=10).mean()
    auc = auc_score(model, urm_validation, item_features=icm_matrix).mean()

    # Print performance
    print(f"Epoch {epoch}/{num_epochs} completed")
    print(f"Precision@10: {precision:.4f}, AUC: {auc:.4f}")


Epoch 1/20 completed
Epoch 2/20 completed
Epoch 3/20 completed
Epoch 4/20 completed
Epoch 5/20 completed
Epoch 6/20 completed
Epoch 7/20 completed
Epoch 8/20 completed
Epoch 9/20 completed
Epoch 10/20 completed
Epoch 11/20 completed
Epoch 12/20 completed
Epoch 13/20 completed
Epoch 14/20 completed
Epoch 15/20 completed
Epoch 16/20 completed
Epoch 17/20 completed
Epoch 18/20 completed
Epoch 19/20 completed
Epoch 20/20 completed


## HyperParams Optimization

In [22]:
import optuna
from Evaluation.Evaluator import EvaluatorHoldout

# Define the number of trials
n_trials = 30

# Define the evaluator for validation
evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 1909 ( 5.3%) Users that have less than 1 test interactions


In [ ]:
import matplotlib.pyplot as plt
from lightfm.evaluation import precision_at_k, auc_score
from lightfm import LightFM

# Initialize the model
model = LightFM(loss='warp', no_components=30)

# Number of epochs
num_epochs = 20

# Storage for metrics
precision_scores = []
auc_scores = []

# Training loop with performance tracking
for epoch in range(1, num_epochs + 1):
    # Partial fit for one epoch
    model.fit_partial(
        urm_train,
        item_features=icm_matrix,
        num_threads=4
    )
    
    # Evaluate the model
    precision = precision_at_k(model, urm_validation, item_features=icm_matrix, k=10).mean()
    auc = auc_score(model, urm_validation, item_features=icm_matrix).mean()

    # Store metrics
    precision_scores.append(precision)
    auc_scores.append(auc)

    # Print performance
    print(f"Epoch {epoch}/{num_epochs} completed")
    print(f"Precision@10: {precision:.4f}, AUC: {auc:.4f}")

# Plotting the metrics
plt.figure(figsize=(12, 6))
plt.plot(range(1, num_epochs + 1), precision_scores, label="Precision@10")
plt.plot(range(1, num_epochs + 1), auc_scores, label="AUC")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.title("Performance Metrics Across Epochs")
plt.legend()
plt.grid()
plt.show()


In [15]:
urm_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1235224 stored elements and shape (35736, 38121)>

In [ ]:
import optuna
from optuna.logging import set_verbosity, INFO
from lightfm import LightFM
from Evaluation.Evaluator import EvaluatorHoldout

# Define the objective function
def objective(trial):
    # Suggest hyperparameters
    loss_function = trial.suggest_categorical("loss", ["logistic", "bpr", "warp", "warp-kos"])
    no_components = trial.suggest_int("no_components", 10, 200)  # Latent factors
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    item_alpha = trial.suggest_float("item_alpha", 1e-6, 1e-1, log=True)  # L2 regularization for items
    user_alpha = trial.suggest_float("user_alpha", 1e-6, 1e-1, log=True)  # L2 regularization for users

    # Initialize the LightFM model with suggested hyperparameters
    model = LightFM(
        loss=loss_function,
        no_components=no_components,
        learning_rate=learning_rate,
        item_alpha=item_alpha,
        user_alpha=user_alpha,
    )
    
    # Fit the model
    model.fit(
        urm_train,
        item_features=icm_matrix,
        epochs=10,
        num_threads=4  # Adjust based on your system
    )
    
    # Use EvaluatorHoldout to calculate MAP@10
    results_df, _ = evaluator_validation.evaluateRecommender(model)
    map_at_10 = results_df.loc[10]["MAP"]  # Extract MAP@10 from results

    return map_at_10

# Progress callback to track optimization progress
def progress_callback(study, trial):
    print(f"Trial {trial.number} finished with value: {trial.value} and parameters: {trial.params}")

# Set Optuna verbosity for detailed logging
set_verbosity(INFO)

# Create and run the Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50, callbacks=[progress_callback])

# Output the best hyperparameters found
print("Best hyperparameters:", study.best_params)
print("Best MAP@10:", study.best_value)


[I 2024-11-21 14:51:17,153] A new study created in memory with name: no-name-acf8c47c-fc02-41cb-9cd2-9c4502617e0e


In [29]:
# Predict scores for all items for a specific user
user_id = 0
scores = model.predict(user_id, np.arange(num_items), item_features=icm_matrix)

# Rank items by score
top_items = np.argsort(-scores)[:10]  # Top 10 items
print(f"Recommended items for user {user_id}: {top_items}")


Recommended items for user 0: [ 3730 11549 15084 12723  4329  6194 30120 20895 30716  9241]


In [57]:
from scipy.sparse import coo_matrix

# Assuming URM is a DataFrame with columns: user_id, item_id, data
URM_sparse = coo_matrix(
    (URM['data'].values, (URM['user_id'].values, URM['item_id'].values))
)

In [58]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

# Assuming URM_all is already loaded as URM
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage=0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage=0.8)


AttributeError: 'DataFrame' object has no attribute 'nnz'

In [ ]:
from lightfm import LightFM

# Initialize LightFM model
model = LightFM(loss='warp', no_components=30)

# Training loop with epoch progress tracking
num_epochs = 20
for epoch in range(1, num_epochs + 1):
    model.fit_partial(
        URM_train,
        item_features=icm_matrix,
        num_threads=4  # Multithreading for faster training
    )
    print(f"Epoch {epoch}/{num_epochs} completed")


In [ ]:
import numpy as np

def lightfm_recommend_batch(model, user_ids, URM_train, item_features, at=10):
    """
    Generate top-N recommendations for a batch of users using LightFM.
    
    Args:
        model: Trained LightFM model.
        user_ids: List of user IDs for which to generate recommendations.
        URM_train: Training matrix to exclude seen items.
        item_features: Optional LightFM item feature matrix.
        at: Number of recommendations to generate.

    Returns:
        List of lists containing recommended item IDs for each user.
    """
    num_items = URM_train.shape[1]
    recommendations = []
    
    for user_id in user_ids:
        scores = model.predict(user_id, np.arange(num_items), item_features=item_features)
        
        # Exclude items seen in training
        seen_items = URM_train[user_id].indices
        scores[seen_items] = -np.inf
        
        # Get top-N recommendations
        top_items = np.argsort(-scores)[:at]
        recommendations.append(top_items)
    
    return recommendations


In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout

# Initialize evaluator for validation
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

# Define a wrapper for the evaluator to use LightFM predictions
class LightFMRecommenderWrapper:
    def __init__(self, model, URM_train, item_features):
        self.model = model
        self.URM_train = URM_train
        self.item_features = item_features

    def recommend(self, user_id, at=10):
        scores = self.model.predict(user_id, np.arange(self.URM_train.shape[1]), item_features=self.item_features)
        seen_items = self.URM_train[user_id].indices
        scores[seen_items] = -np.inf
        return np.argsort(-scores)[:at]

# Wrap the LightFM model
lightfm_wrapper = LightFMRecommenderWrapper(model, URM_train, icm_matrix)

# Evaluate the wrapped model
results = evaluator_validation.evaluateRecommender(lightfm_wrapper)
print("Validation Results:", results)


# EXTRA

In [13]:
import pandas as pd

# Load the target users
target_users = pd.read_csv("./data/data_target_users_test.csv")

# Create predictions for each user
# Assuming `recommender` is your fitted model and `top_k` is the number of recommendations per user
top_k = 10

# Generate recommendations
def generate_recommendations(user_id):
    # Generate recommendations for the user, and slice to take top_k items
    recommendations = p3opt.recommend(user_id)[:top_k]
    return " ".join(map(str, recommendations))

# Apply the recommendation function to each user in the target list
target_users['item_list'] = target_users['user_id'].apply(generate_recommendations)

# Save to CSV
target_users.to_csv("submission.csv", index=False)


In [ ]:
urm_train

In [ ]:
urm_validation

In [ ]:
urm_test